<a href="https://colab.research.google.com/github/olaviinha/NeuralTextToImage/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Stable Diffusion <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Neural text-to-image</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralTextToImage" target="_blank"><font color="#999" size="4">Github</font></a>

Generate images from text prompt using [Stable Diffusion](https://github.com/huggingface/diffusers). Stable Diffusion is a text-to-image latent diffusion model created by the researchers and engineers from [CompVis](https://github.com/CompVis), [Stability AI](https://stability.ai) and [LAION](https://laion.ai).

**Requirements** to run this notebook:
- [Hugging Face](https://huggingface.co/) user account (register for free).
- Agreeing to the terms of [v1-4 model card](https://huggingface.co/CompVis/stable-diffusion-v1-4).
- Hugging Face access token, which can be found [here](https://huggingface.co/settings/tokens) when you have a user account.

**Tips:**
- You may use multiple prompts in one run by separating them with `;`.


In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>


force_setup = False
pip_packages = 'diffusers==0.2.4 transformers scipy ftfy'
main_repository = ''

#@markdown Copy-paste your Hugging Face access token in the field below prior to executing this cell. You can find your access token [here](https://huggingface.co/settings/tokens).
access_token = "" #@param {type:"string"}

mount_drive = True #@param {type:"boolean"}




# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive is True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if main_repository is not '':
  !git clone {main_repository}

import time, sys
from datetime import timedelta

# !pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
# !pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip

!pip install "ipywidgets>=7,<8"

# output.enable_custom_widget_manager()
# from huggingface_hub import notebook_login
# notebook_login()


from PIL import Image

def image_grid(imgs, rows, cols):
  assert len(imgs) == rows*cols

  w, h = imgs[0].size
  grid = Image.new('RGB', size=(cols*w, rows*h))
  grid_w, grid_h = grid.size
  
  for i, img in enumerate(imgs):
    grid.paste(img, box=(i%cols*w, i//cols*h))
  return grid
  

import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=access_token)

pipe = pipe.to("cuda")

# Disable NSFW filter
# pipe.safety_checker = lambda images, **kwargs: (images, False)


from torch import autocast


output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:

#@markdown <br>

#@markdown #S̛̞̩͎͓ ̦̤͉͚̏ ̧̠͋͘ͅl͕̞͕̝͗̐͘.̠̰̳̫̈́̚ ̡͉̼̩̬̈́̇͒͘ȩ̨͎͛̔͆͊̏͜ͅ.͕̩̹̠̕͜ ̛̦̦̮e̢͐͊͂̀̊ͅ ̜̙̝̊͋ ̬̝̱̱͗p̮̎̽̌

#@markdown <br>

generate_image_of = "" #@param {type:"string"}
seed = 0 #@param {type:"integer"}
batch_size = 1 #@param {type:"integer"}
repeats = 1 #@param {type:"integer"}
output_dir = "" #@param {type:"string"}


uniq_id = gen_id()

# Input
# input_is_dir = False
# if os.path.isdir(drive_root+input) or '*' in input:
#   input_is_dir = True
#   if not '*' in input:
#     input = drive_root+fix_path(input)
#     dir_in = input
#     inputs = glob(dir_in+'/*')
#   else:
#     inputs = glob(drive_root+input)

# elif os.path.isfile(drive_root+input):
#   input = drive_root+input
#   dir_in = path_dir(input)
# else:
#   op(c.fail, 'Input not found')
#   sys.exit('Input not found')

# Output
if output_dir == '':
  dir_out = dir_tmp
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()

if generate_image_of is 'pre':
  prompts = predefined_prompts
else:
  if ';' in generate_image_of:
    prompts = [x.strip() for x in generate_image_of.split(';')]
  else:
    prompts = [generate_image_of]

if repeats > 1:
  prompts = prompts * repeats

# -- DO THINGS --


if seed is 0:
  seed = int(time.time())

generator = torch.Generator("cuda").manual_seed(seed)
total = len(prompts)

for i, prompt in enumerate(prompts):
  n = i+1
  op(c.title, str(n)+'/'+str(total)+' Generating:', prompt, time=True)
  img_row = []
  for i in range(batch_size):
    with autocast("cuda"):
      # images = pipe(prompt, generator=generator)["sample"][0]
      images = pipe(prompt, generator=generator)["sample"]
    img_row.extend(images)
    for image in images:
      file_out = uniq_id+'_'+slug(prompt)[:50]+'_'+str(i).zfill(3)+'.png'
      image.save(dir_out+file_out)
  grid = image_grid(img_row, rows=1, cols=batch_size)
  display(grid)
  # or if you're in a google colab you can directly display it with 
  # image

op(c.ok, 'Images saved in', dir_out.replace(drive_root, ''))

timer_end = time.time()

print('\nElapsed', timedelta(seconds=timer_end-timer_start))
